In [1]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

# Pull Pato Yield Data

In [2]:
# Wafer Level df

daysback = 300
starttime = time.time()
Query = """
WITH lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND B.FAMILY_CODE = 'Q5'
        AND EWR_NAME IN ('7HPP_Name_P10')
        AND CELL_ID LIKE '7HPP_Name_P10%'
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
),
pato_cte AS(
    SELECT lot_id, wafer_id, family_code, MIN(tw.Last_test_date) AS Pato_date, 1 AS pato_n,
      AVG(CASE WHEN parm_Label = 'PATOALL_H2_HL_PerfectYield' THEN yield ELSE NULL END) AS pato_all_vmax_py,
      AVG(CASE WHEN parm_Label = 'PATOALL_H2_NL_PerfectYield' THEN yield ELSE NULL END) AS pato_all_vnom_py,
      AVG(CASE WHEN parm_Label = 'PATOALL_H2_LL_PerfectYield' THEN yield ELSE NULL END) AS pato_all_vmin_py,
      AVG(CASE WHEN parm_Label = 'PATO1_H2_HL_PerfectYield' THEN yield ELSE NULL END) AS pato1_vmax_py,
      AVG(CASE WHEN parm_Label = 'PATO1_H2_NL_PerfectYield' THEN yield ELSE NULL END) AS pato1_vnom_py,
      AVG(CASE WHEN parm_Label = 'PATO1_H2_LL_PerfectYield' THEN yield ELSE NULL END) AS pato1_vmin_py,
      AVG(CASE WHEN parm_Label = 'PATO2_H2_HL_PerfectYield' THEN yield ELSE NULL END) AS pato2_vmax_py,
      AVG(CASE WHEN parm_Label = 'PATO2_H2_NL_PerfectYield' THEN yield ELSE NULL END) AS pato2_vnom_py,
      AVG(CASE WHEN parm_Label = 'PATO2_H2_LL_PerfectYield' THEN yield ELSE NULL END) AS pato2_vmin_py
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testParmKey = tp.testParmKey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE Last_test_date >= (current date - {} days)
      AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('PTO11', 'PTO21')
      AND weighted_Mean IS NOT null AND abs(weighted_Mean) < 1e25
      AND ( ucase(tp.parm_Label) like '%PERFECTYIELD' ) 
    GROUP BY lot_id, wafer_id, family_code
)
SELECT ln.lot_name, pato.wafer_id, family_code,
    Pato_date, Pato_n, pato_all_vmax_py, pato_all_vnom_py, pato_all_vmin_py,
    pato1_vmax_py, pato1_vnom_py, pato1_vmin_py, pato2_vmax_py, pato2_vnom_py, pato2_vmin_py
FROM pato_cte pato
LEFT JOIN lot_name_cte ln ON pato.wafer_id = ln.wafer_id
ORDER BY pato_date desc, pato.wafer_id
""".format(daysback,daysback,daysback)

w_df = ibmdata.isdw.query(Query)

w_df['pato_all_vmax_py'] = w_df['pato_all_vmax_py'].astype(float).round(2)
w_df['pato_all_vnom_py'] = w_df['pato_all_vnom_py'].astype(float).round(2)
w_df['pato_all_vmin_py'] = w_df['pato_all_vmin_py'].astype(float).round(2)
w_df['pato1_vmax_py'] = w_df['pato1_vmax_py'].astype(float).round(2)
w_df['pato1_vnom_py'] = w_df['pato1_vnom_py'].astype(float).round(2)
w_df['pato1_vmin_py'] = w_df['pato1_vmin_py'].astype(float).round(2)
w_df['pato2_vmax_py'] = w_df['pato2_vmax_py'].astype(float).round(2)
w_df['pato2_vnom_py'] = w_df['pato2_vnom_py'].astype(float).round(2)
w_df['pato2_vmin_py'] = w_df['pato2_vmin_py'].astype(float).round(2)

print(time.time()-starttime)

1.3615000247955322


In [28]:
w_df

,lot_name,wafer_id,family_code,pato_date,pato_n,pato_all_vmax_py,pato_all_vnom_py,pato_all_vmin_py,pato1_vmax_py,pato1_vnom_py,pato1_vmin_py,pato2_vmax_py,pato2_vnom_py,pato2_vmin_py
0,CS13,AZ725-01,Q5,2020-08-09,1,100.00,100.00,53.33,100.00,100.00,69.04,100.00,100.00,65.11
1,CS13,AZ725-02,Q5,2020-08-09,1,100.00,100.00,77.33,100.00,100.00,89.28,100.00,100.00,84.88
2,CS13,AZ725-03,Q5,2020-08-09,1,97.33,97.33,69.33,100.00,100.00,79.76,97.67,97.67,76.74
3,CS13,AZ725-04,Q5,2020-08-09,1,98.66,98.66,80.00,98.80,98.80,88.09,98.83,98.83,80.23
4,CS13,AZ725-05,Q5,2020-08-09,1,98.66,98.66,52.00,100.00,100.00,58.33,98.83,98.83,61.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Lot2,AZ6PK-22,Q5,2020-06-29,1,100.00,100.00,96.00,98.82,98.82,96.47,100.00,100.00,94.18
197,Lot2,AZ6PK-23,Q5,2020-06-29,1,100.00,100.00,97.33,98.82,98.82,98.82,100.00,100.00,95.34
198,Lot2,AZ6PK-24,Q5,2020-06-29,1,100.00,100.00,96.00,98.82,98.82,96.47,100.00,100.00,95.34
199,Lot2,AZ6PK-25,Q5,2020-06-29,1,100.00,100.00,94.66,98.82,98.82,97.64,100.00,100.00,94.18


# Pull LL1 Yields

In [3]:
Query = """
SELECT lot_Id, wafer_Id, parm_Label as parameter, SUBSTRING(parm_Label,17,100) AS Label, LEFT(parm_label,5) AS pato,
    yield, 100-yield AS yld_loss
FROM DMIW.PTileWaferFact ptwf
INNER JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = ptwf.testParmKey
INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = ptwf.testedWaferKey
WHERE Last_test_date >= (current date - 300 days) and Tech_id = '7HPP' and tw.Calcdefs in ('PTO11', 'PTO21')
    AND weighted_Mean is not null and abs(weighted_Mean) < 1e25
    AND ( UCASE(tp.parm_Label) LIKE '%CH_LL1%' )
Order BY lot_Id, wafer_Id, parm_Label, level
"""

LL1_df = ibmdata.isdw.query(Query)
LL1_df['yield'] = LL1_df['yield'].astype(float)
LL1_df['yld_loss'] = LL1_df['yld_loss'].astype(float)

In [7]:
LL1_df

,lot_id,wafer_id,parameter,label,pato,yield,yld_loss
0,AZ6B5.1,AZ6B5-19,Pato1~H2~CH_LL1_A100CFG_FailCount,A100CFG_FailCount,Pato1,100.00,0.00
1,AZ6B5.1,AZ6B5-19,Pato1~H2~CH_LL1_A101CFG_FailCount,A101CFG_FailCount,Pato1,100.00,0.00
2,AZ6B5.1,AZ6B5-19,Pato1~H2~CH_LL1_A10_CFG_FailCount,A10_CFG_FailCount,Pato1,100.00,0.00
3,AZ6B5.1,AZ6B5-19,Pato1~H2~CH_LL1_B100VTL_FailCount,B100VTL_FailCount,Pato1,100.00,0.00
4,AZ6B5.1,AZ6B5-19,Pato1~H2~CH_LL1_B101VTL_FailCount,B101VTL_FailCount,Pato1,100.00,0.00
...,...,...,...,...,...,...,...
64882,AZ725.1,AZ725-25,Pato2~H2~CH_LL1_G10_X13_FailCount,G10_X13_FailCount,Pato2,98.82,1.18
64883,AZ725.1,AZ725-25,Pato2~H2~CH_LL1_G10_X14_FailCount,G10_X14_FailCount,Pato2,100.00,0.00
64884,AZ725.1,AZ725-25,Pato2~H2~CH_LL1_G10_X15_FailCount,G10_X15_FailCount,Pato2,98.82,1.18
64885,AZ725.1,AZ725-25,Pato2~H2~CH_LL1_G10_X16_FailCount,G10_X16_FailCount,Pato2,100.00,0.00


# Pivot LL1 Data

In [4]:
pivot_df = LL1_df.pivot(index = ['wafer_id'], columns='parameter', values='yield' ).reset_index()

### Keep Only The Rows We Want From Pato Yields

In [5]:
temp_df = w_df[['lot_name','wafer_id']]
temp_df['pato1_delta'] = w_df['pato1_vnom_py'] - w_df['pato1_vmin_py']
#temp_df['pato1_delta'] = w_df['pato1_vmax_py'] - w_df['pato1_vmin_py']
df = temp_df.merge(pivot_df,on='wafer_id',how='inner')

In [6]:
# One hot encode
columns = df.columns[3:]
for col in columns:
    df[col] = df[col].apply(lambda x: 0 if float(x) < 100.00 else 1)

In [27]:
df.to_csv('~/Downloads/patoVminDelta.csv')

# Analysis

### Find the Lots/Wafers with the Largest Vmax Vmin Delta

In [7]:
wafers = df.sort_values('pato1_delta',ascending=False)[0:50].wafer_id

In [8]:
df.sort_values('pato1_delta',ascending=False)[0:50]

,lot_name,wafer_id,pato1_delta,Pato1~H2~CH_LL1_A100CFG_FailCount,Pato1~H2~CH_LL1_A101CFG_FailCount,Pato1~H2~CH_LL1_A10_CFG_FailCount,Pato1~H2~CH_LL1_B100VTL_FailCount,Pato1~H2~CH_LL1_B101VTL_FailCount,Pato1~H2~CH_LL1_B10_VTL_FailCount,Pato1~H2~CH_LL1_C100PLL_FailCount,Pato1~H2~CH_LL1_C101PLL_FailCount,Pato1~H2~CH_LL1_C10_PLL_FailCount,Pato1~H2~CH_LL1_D10_PLL_FailCount,Pato1~H2~CH_LL1_F100X01_FailCount,Pato1~H2~CH_LL1_F100X02_FailCount,Pato1~H2~CH_LL1_F100X03_FailCount,Pato1~H2~CH_LL1_F100X04_FailCount,Pato1~H2~CH_LL1_F100X05_FailCount,Pato1~H2~CH_LL1_F100X06_FailCount,Pato1~H2~CH_LL1_F100X07_FailCount,Pato1~H2~CH_LL1_F100X08_FailCount,Pato1~H2~CH_LL1_F100X09_FailCount,Pato1~H2~CH_LL1_F100X10_FailCount,Pato1~H2~CH_LL1_F100X11_FailCount,Pato1~H2~CH_LL1_F100X12_FailCount,Pato1~H2~CH_LL1_F100X13_FailCount,Pato1~H2~CH_LL1_F100X14_FailCount,Pato1~H2~CH_LL1_F100X15_FailCount,Pato1~H2~CH_LL1_F100X16_FailCount,Pato1~H2~CH_LL1_F100X17_FailCount,Pato1~H2~CH_LL1_F101X01_FailCount,Pato1~H2~CH_LL1_F101X02_FailCount,Pato1~H2~CH_LL1_F101X03_FailCount,Pato1~H2~CH_LL1_F101X04_FailCount,Pato1~H2~CH_LL1_F101X05_FailCount,Pato1~H2~CH_LL1_F101X06_FailCount,Pato1~H2~CH_LL1_F101X07_FailCount,Pato1~H2~CH_LL1_F101X08_FailCount,Pato1~H2~CH_LL1_F101X09_FailCount,Pato1~H2~CH_LL1_F101X10_FailCount,Pato1~H2~CH_LL1_F101X11_FailCount,Pato1~H2~CH_LL1_F101X12_FailCount,Pato1~H2~CH_LL1_F101X13_FailCount,Pato1~H2~CH_LL1_F101X14_FailCount,Pato1~H2~CH_LL1_F101X15_FailCount,Pato1~H2~CH_LL1_F101X16_FailCount,Pato1~H2~CH_LL1_F101X17_FailCount,Pato1~H2~CH_LL1_F10LX01_FailCount,Pato1~H2~CH_LL1_F10LX02_FailCount,Pato1~H2~CH_LL1_F10LX03_FailCount,Pato1~H2~CH_LL1_F10LX04_FailCount,Pato1~H2~CH_LL1_F10LX05_FailCount,Pato1~H2~CH_LL1_F10LX06_FailCount,Pato1~H2~CH_LL1_F10LX07_FailCount,Pato1~H2~CH_LL1_F10LX08_FailCount,Pato1~H2~CH_LL1_F10LX09_FailCount,Pato1~H2~CH_LL1_F10LX10_FailCount,Pato1~H2~CH_LL1_F10LX11_FailCount,Pato1~H2~CH_LL1_F10LX12_FailCount,Pato1~H2~CH_LL1_F10LX13_FailCount,Pato1~H2~CH_LL1_F10LX14_FailCount,Pato1~H2~CH_LL1_F10LX15_FailCount,Pato1~H2~CH_LL1_F10LX16_FailCount,Pato1~H2~CH_LL1_F10LX17_FailCount,Pato1~H2~CH_LL1_G100X01_FailCount,Pato1~H2~CH_LL1_G100X02_FailCount,Pato1~H2~CH_LL1_G100X03_FailCount,Pato1~H2~CH_LL1_G100X04_FailCount,Pato1~H2~CH_LL1_G100X05_FailCount,Pato1~H2~CH_LL1_G100X06_FailCount,Pato1~H2~CH_LL1_G100X07_FailCount,Pato1~H2~CH_LL1_G100X08_FailCount,Pato1~H2~CH_LL1_G100X09_FailCount,Pato1~H2~CH_LL1_G100X10_FailCount,Pato1~H2~CH_LL1_G100X11_FailCount,Pato1~H2~CH_LL1_G100X12_FailCount,Pato1~H2~CH_LL1_G100X13_FailCount,Pato1~H2~CH_LL1_G100X14_FailCount,Pato1~H2~CH_LL1_G100X15_FailCount,Pato1~H2~CH_LL1_G100X16_FailCount,Pato1~H2~CH_LL1_G100X17_FailCount,Pato1~H2~CH_LL1_G101X01_FailCount,Pato1~H2~CH_LL1_G101X02_FailCount,Pato1~H2~CH_LL1_G101X03_FailCount,Pato1~H2~CH_LL1_G101X04_FailCount,Pato1~H2~CH_LL1_G101X05_FailCount,Pato1~H2~CH_LL1_G101X06_FailCount,Pato1~H2~CH_LL1_G101X07_FailCount,Pato1~H2~CH_LL1_G101X08_FailCount,Pato1~H2~CH_LL1_G101X09_FailCount,Pato1~H2~CH_LL1_G101X10_FailCount,Pato1~H2~CH_LL1_G101X11_FailCount,Pato1~H2~CH_LL1_G101X12_FailCount,Pato1~H2~CH_LL1_G101X13_FailCount,Pato1~H2~CH_LL1_G101X14_FailCount,Pato1~H2~CH_LL1_G101X15_FailCount,Pato1~H2~CH_LL1_G101X16_FailCount,Pato1~H2~CH_LL1_G101X17_FailCount,Pato1~H2~CH_LL1_G10AX01_FailCount,Pato1~H2~CH_LL1_G10AX02_FailCount,Pato1~H2~CH_LL1_G10AX03_FailCount,Pato1~H2~CH_LL1_G10AX04_FailCount,Pato1~H2~CH_LL1_G10AX05_FailCount,Pato1~H2~CH_LL1_G10AX06_FailCount,Pato1~H2~CH_LL1_G10AX07_FailCount,Pato1~H2~CH_LL1_G10AX08_FailCount,Pato1~H2~CH_LL1_G10AX09_FailCount,Pato1~H2~CH_LL1_G10AX10_FailCount,Pato1~H2~CH_LL1_G10AX11_FailCount,Pato1~H2~CH_LL1_G10AX12_FailCount,Pato1~H2~CH_LL1_G10AX13_FailCount,Pato1~H2~CH_LL1_G10AX14_FailCount,Pato1~H2~CH_LL1_G10AX15_FailCount,Pato1~H2~CH_LL1_G10AX16_FailCount,Pato1~H2~CH_LL1_G10AX17_FailCount,Pato1~H2~CH_LL1_G10_X01_FailCount,Pato1~H2~CH_LL1_G10_X02_FailCount,Pato1~H2~CH_LL1_G10_X03_FailCount,Pato1~H2~CH_LL1_G10_X04_FailCount,Pato1~H2~CH_LL1_G10_X05_F

### Function to find the Pareto for a set of wafers or lots

In [9]:
def plot_pareto(df,n=10,yld_loss = True, wafers = None, lots = None, alldata = True):
    df = df[df['pato'].isin(['Pato1'])]

    # This logic deals with toggling between yield and yld_loss
    if yld_loss == True:
        yparm = 'yld_loss'
        asc = False
    else:
        yparm = 'yield'
        asc = True
    
    # This logic is to filter by wafer, lot, or alldata
    if not wafers or type(wafers) != list:
        if not lots or type(lots) != list:
            if alldata == False or type(alldata) != bool:
                print('Please enter a WaferList, LotList or set alldata to True')
                return
            else:
                inclusiontxt = 'all wafers'
                pass
        else:
            df = df[df['lot_id'].isin(lots)]
            alldata = False
            if len(lots) > 3:
                inclusiontxt = 'lots in '+', '.join(lots[0:3])+'...'
            else:
                inclusiontxt = 'lots in '+', '.join(lots)
    else:
        df = df[df['wafer_id'].isin(wafers)]
        alldata = False
        if len(wafers) > 3:
            inclusiontxt = 'lots in '+', '.join(wafers[0:3])+'...'
        else:
            inclusiontxt = 'lots in '+', '.join(wafers)
        
    num_wafers = df['wafer_id'].nunique()
    
    # This part of the code deals with plotting the pareto     
    df_agg = df.groupby(['label']).mean().sort_values(yparm,ascending=asc).reset_index()[0:n]
    
    fig = px.bar(df_agg, x='label', y=yparm,
            hover_data=['label', yparm],
            labels={'label':'Label',yparm:yparm+' (%)'})
    fig.update_layout(title_text='Vmin LL1 pareto for '+inclusiontxt+' ({} distinct wafers)'.format(num_wafers))
    fig.update_layout(xaxis_categoryorder = 'total descending')
    fig.show()

In [12]:
plot_pareto(LL1_df,n=40, lots=['AZ73F.1'])

In [35]:
plot_pareto(LL1_df,n=40,wafers = list(wafers))